<a href="https://colab.research.google.com/github/bacoco/LLM_train/blob/main/Fine_tune_Instruct_LLMs_with_ORPO_Example_with_Mistral_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook fine-tune and align Mistral with ORPO.

The last section of the notebook also shows an example of ORPO training with [GaLore](https://kaitchup.substack.com/p/galore-full-fine-tuning-on-your-gpu).




First, we need all these dependencies:

In [ ]:
!pip install -q -U bitsandbytes
!pip install --upgrade -q -U transformers
!pip install -q -U peft
!pip install -q -U accelerate
!pip install -q -U datasets
!pip install -q -U git+https://github.com/huggingface/trl.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━

Import all the necessary packages.

In [ ]:
import torch, multiprocessing
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import ORPOTrainer, ORPOConfig

Load the tokenizer and configure padding

In [ ]:
major_version, minor_version = torch.cuda.get_device_capability()
if major_version >= 8:
  !pip install flash-attn
  torch_dtype = torch.bfloat16
  attn_implementation='flash_attention_2'
  print("Your GPU is compatible with FlashAttention and bfloat16.")
else:
  torch_dtype = torch.float16
  attn_implementation='eager'
  print("Your GPU is not compatible with FlashAttention and bfloat16.")

model_name = "mistralai/Mistral-7B-v0.1"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left' #Necessary for FlashAttention compatibility

Your GPU is compatible with FlashAttention and bfloat16.


Load the ultrafeedback dataset prepared by Hugging Face for preference optimization. I apply a chat template to stringify the JSON.

In [ ]:
dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split=["train_prefs","test_prefs"])

def process(row):
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

dataset[0] = dataset[0].map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

dataset[1] = dataset[1].map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

print(dataset)

Map (num_proc=16):   0%|          | 0/61135 [00:00<?, ? examples/s]


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using

Map (num_proc=16):   0%|          | 0/2000 [00:00<?, ? examples/s]


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using

[Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 61135
}), Dataset({
    features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
    num_rows: 2000
})]


Load the model and prepare it for QLoRA fine-tuning.

In [ ]:
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, torch_dtype=torch_dtype, quantization_config=bnb_config, device_map={"": 0},  attn_implementation=attn_implementation
)
model = prepare_model_for_kbit_training(model)
#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Define the configuration of LoRA

In [ ]:
peft_config = LoraConfig(
        lora_alpha=16,
        lora_dropout=0.05,
        r=16,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"]
)

For this tutorial, I trained for only 100 steps.
If you want to speed up training, disable the evaluation. It takes around 1.5 hours to evaluate a checkpoint on the test split.

In [ ]:
orpo_config = ORPOConfig(
    output_dir="./results/",
    evaluation_strategy="steps",
    do_eval=True,
    optim="paged_adamw_8bit",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=2,
    log_level="debug",
    logging_steps=20,
    learning_rate=8e-6,
    eval_steps=20,
    max_steps=100,
    save_steps=20,
    save_strategy='epoch',
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    beta=0.1, #beta is ORPO's lambda
    max_length=1024,
)

trainer = ORPOTrainer(
        model=model,
        train_dataset=dataset[0],
        eval_dataset=dataset[1],
        peft_config=peft_config,
        args=orpo_config,
        tokenizer=tokenizer,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:218: UserWarning: `max_prompt_length` is not set in the ORPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:247: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sample

Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
20,1.016000,1.019213,5743.114300,0.348000,0.174000,-0.098439,-0.106847,0.574500,0.008407,-1.068466,-0.984393,-2.616019,-2.698306,0.952289,-0.669238,0.125191
40,1.000500,0.978686,5743.275300,0.348000,0.174000,-0.092994,-0.100765,0.568500,0.007771,-1.007654,-0.929941,-2.633790,-2.714040,0.911652,-0.670338,0.120774
60,0.962400,0.951861,5737.338100,0.349000,0.174000,-0.089386,-0.096861,0.573500,0.007475,-0.968612,-0.893857,-2.627004,-2.706775,0.884787,-0.670742,0.120029


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 2


Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
20,1.016000,1.019213,5743.114300,0.348000,0.174000,-0.098439,-0.106847,0.574500,0.008407,-1.068466,-0.984393,-2.616019,-2.698306,0.952289,-0.669238,0.125191
40,1.000500,0.978686,5743.275300,0.348000,0.174000,-0.092994,-0.100765,0.568500,0.007771,-1.007654,-0.929941,-2.633790,-2.714040,0.911652,-0.670338,0.120774
60,0.962400,0.951861,5737.338100,0.349000,0.174000,-0.089386,-0.096861,0.573500,0.007475,-0.968612,-0.893857,-2.627004,-2.706775,0.884787,-0.670742,0.120029
80,0.926200,0.932171,5734.474100,0.349000,0.174000,-0.086718,-0.093974,0.566000,0.007256,-0.939740,-0.867178,-2.620364,-2.699521,0.865035,-0.671352,0.119524
100,0.947900,0.923387,5735.446500,0.349000,0.174000,-0.085499,-0.092642,0.565500,0.007143,-0.926419,-0.854992,-2.618192,-2.697114,0.856202,-0.671856,0.119217


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 2
Saving model checkpoint to ./results/checkpoint-100
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/26bca36bde8333b5d7f72e9ed20ccda6a618af24/config.json
Model config MistralConfig {
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.39.3",
  "use_cache": true,
  "vocab_size": 32000
}

tokenizer config file saved in ./results/checkpoint-100/tokenizer_config.jso

TrainOutput(global_step=100, training_loss=0.9706012344360352, metrics={'train_runtime': 35278.7723, 'train_samples_per_second': 0.023, 'train_steps_per_second': 0.003, 'total_flos': 0.0, 'train_loss': 0.9706012344360352, 'epoch': 0.01})

# Bonus section: ORPO with GaLore

The following cells runs ORPO training with GaLore. It requires almost 40 GB.

More about GaLore here:
[GaLore: Full Fine-tuning on Your GPU](https://kaitchup.substack.com/p/galore-full-fine-tuning-on-your-gpu)


In [ ]:
!pip install git+https://github.com/jiaweizzhao/GaLore

  Cloning https://github.com/jiaweizzhao/GaLore to /tmp/pip-req-build-3m9b2iyo
  Running command git clone --filter=blob:none --quiet https://github.com/jiaweizzhao/GaLore /tmp/pip-req-build-3m9b2iyo
  Resolved https://github.com/jiaweizzhao/GaLore to commit 1b36c33782bdd74a4d6a4f51bc626ef67f51011f
  Preparing metadata (setup.py) ... done
  Created wheel for galore-torch: filename=galore_torch-1.0-py3-none-any.whl size=13310 sha256=fd52db2c28d983d55e8c5f2d80cc55422d5e0e9ffad9a1aadcc2b94fc0775b99
  Stored in directory: /tmp/pip-ephem-wheel-cache-kbqfnu9f/wheels/88/47/b5/ca5f75e9f8a2eef76440b7070f8e82f0099831c3d13ebbe221
Successfully built galore-torch


In [ ]:
import torch, multiprocessing
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import ORPOTrainer, ORPOConfig

major_version, minor_version = torch.cuda.get_device_capability()
if major_version >= 8:
  !pip install flash-attn
  torch_dtype = torch.bfloat16
  attn_implementation='flash_attention_2'
  print("Your GPU is compatible with FlashAttention and bfloat16.")
else:
  torch_dtype = torch.float16
  attn_implementation='eager'
  print("Your GPU is not compatible with FlashAttention and bfloat16.")

model_name = "mistralai/Mistral-7B-v0.1"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, add_eos_token=True, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left' #Necessary for FlashAttention compatibility

dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized", split=["train_prefs","test_prefs"])

def process(row):
    row["chosen"] = tokenizer.apply_chat_template(row["chosen"], tokenize=False)
    row["rejected"] = tokenizer.apply_chat_template(row["rejected"], tokenize=False)
    return row

dataset[0] = dataset[0].map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

dataset[1] = dataset[1].map(
    process,
    num_proc= multiprocessing.cpu_count(),
    load_from_cache_file=False,
)

print(dataset)

model = AutoModelForCausalLM.from_pretrained(
          model_name, torch_dtype=torch_dtype, device_map={"": 0},  attn_implementation=attn_implementation
)
model.gradient_checkpointing_enable()

orpo_config = ORPOConfig(
    output_dir="./results_orpo_galore/",
    evaluation_strategy="steps",
    do_eval=True,
    optim="galore_adamw_8bit",
    optim_args="rank=512, update_proj_gap=200, scale=1.8",
    optim_target_modules=[r".*attn.*", r".*mlp.*"],
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=2,
    log_level="debug",
    logging_steps=20,
    learning_rate=8e-6,
    eval_steps=20,
    max_steps=100,
    save_steps=20,
    save_strategy='epoch',
    warmup_ratio=0.1,
    lr_scheduler_type="linear",
    beta=0.1, #beta is ORPO's lambda
    max_length=1024,
)


trainer = ORPOTrainer(
        model=model,
        train_dataset=dataset[0],
        eval_dataset=dataset[1],
        args=orpo_config,
        tokenizer=tokenizer,
)

trainer.train()


Your GPU is compatible with FlashAttention and bfloat16.


Map (num_proc=12):   0%|          | 0/61135 [00:00<?, ? examples/s]


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using

Map (num_proc=12):   0%|          | 0/2000 [00:00<?, ? examples/s]


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using the default template for the LlamaTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.


No chat template is defined for this tokenizer - using

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:218: UserWarning: `max_prompt_length` is not set in the ORPOConfig's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/orpo_trainer.py:247: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sample

Step,Training Loss,Validation Loss,Runtime,Samples Per Second,Steps Per Second,Rewards/chosen,Rewards/rejected,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Logits/rejected,Logits/chosen,Nll Loss,Log Odds Ratio,Log Odds Chosen
20,0.896900,0.831252,321.445900,6.222000,3.111000,-0.072961,-0.079216,0.555000,0.006255,-0.792161,-0.729613,-2.658284,-2.705120,0.763083,-0.681696,0.134226
40,0.803700,0.796020,321.410100,6.223000,3.111000,-0.067904,-0.073738,0.547500,0.005834,-0.737383,-0.679042,-2.469285,-2.509542,0.726005,-0.700146,0.152102
60,0.809500,0.786309,321.726600,6.216000,3.108000,-0.066661,-0.072544,0.551500,0.005883,-0.725438,-0.666612,-2.494958,-2.537989,0.714926,-0.713838,0.165371
80,0.776300,0.779980,321.710800,6.217000,3.108000,-0.066029,-0.071821,0.548000,0.005792,-0.718209,-0.660286,-2.519515,-2.565415,0.708572,-0.714080,0.167683
100,0.800100,0.778907,321.321700,6.224000,3.112000,-0.065912,-0.071690,0.548000,0.005777,-0.716897,-0.659124,-2.522480,-2.567922,0.707478,-0.714294,0.168027


***** Running Evaluation *****
  Num examples = 2000
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 2
***** Running Evaluation *****
  Num examples = 2000
  Batch size = 2
Saving model checkpoint to ./results_orpo_galore/checkpoint-100
Configuration saved in ./results_orpo_galore/checkpoint-100/config.json
Configuration saved in ./results_orpo_galore/checkpoint-100/generation_config.json
The model is bigger than the maximum size per checkpoint (5GB) and is going to be split in 3 checkpoint shards. You can find where each parameters has been saved in the index located at ./results_orpo_galore/checkpoint-100/model.safetensors.index.json.
tokenizer config file saved in ./results_orpo_galore/checkpoint-100/tokenizer_config.json
Special tokens file saved in ./results_orpo_galore/checkpoint-100/special_tokens_map.jso

TrainOutput(global_step=100, training_loss=0.8172926712036133, metrics={'train_runtime': 2505.0851, 'train_samples_per_second': 0.319, 'train_steps_per_second': 0.04, 'total_flos': 0.0, 'train_loss': 0.8172926712036133, 'epoch': 0.01})